In [ ]:
# Importacion de modulos utilizados
import csv


# ruta del archivo csv
''' asegurarse de que se haya importado el archivo csv al
    file system (colgado en el directorio root) de colab cada vez
    que se ingresa a esta notebook (es un entorno efimero) '''

path = './listings.csv'


# Variables que definen el nombre y numero de la columna en el csv
neighbourhood = 'neighbourhood_cleansed', 27
room = 'room_type', 32
bedrooms = 'bedrooms', 36
price = 'price', 39
reviews= 'number_of_reviews', 55

lower_price_per_bedrooms = {}
review_per_neigh = {}
roomtype_per_neigh = {}
price_per_neigh = {}

def ignore_firts_row(cell, attribute):
    ''' ignora la primera celda de una columna que
        contiene el nombre de la misma '''

    if cell != attribute:
        return True

def add_value_to_key(dic, key, value):
    ''' Añade a un diccionario dado las claves y valores
        asociados en forma de lista '''

    if key in dic.keys():
        dic[key].append(value)
  
    else:
        dic[key]=[value]
    
    return dic

def string_to_float(value):
      cleaned = value.replace('$', '').replace(',', '')

      ''' Limpia el formato de un numero de una celda
          en formato string, convirtiendolo a flotante '''

      float_number = float(cleaned)
      return float_number

def avarage(listings):
    ''' Calcula el promedio de una lista '''

    return (sum(listings) / len(listings))

def min_value(listings):
    ''' Obtiene el menor valor de una lista '''

    return min(listings)

def count(key):
    ''' toma una lista, y retorna un diccionario
        que asocia a cada elemento de la lista
        el numero de veces que este aparece en la lista '''

    return dict((i, key.count(i)) for i in set(key))

def create_dict(dictio, independent, dependt, comverter=None, avarage=None, min=None, counter=None):
    ''' Toma un diccionario vacio, y dos tuplas (el primer elemento
        es el nombre de la columna, el segundo el numero de la misma):
          - independent: Son los datos que formaran las claves del diccionario
          - dependt: Son los datos que se almacenan en listas asociadas a cada clave '''

    with open(path, newline='') as File:
        reader = csv.reader(File, delimiter=',')

        for row in reader:
          
             if ignore_firts_row(row[dependt[1]], dependt[0]):
                # ignora el nombre la columna, y crea una variable con el valor actual de la celda

                 if comverter == None:
                    element=row[dependt[1]]

                 else:
                    element=comverter(row[dependt[1]])


             if ignore_firts_row(row[independent[1]], independent[0]):
                # Ignora el nombre la columna, y añade la variable a listas asociadas a cada clave

                 add_value_to_key(dictio, row[independent[1]], element)
        

        ''' Condicionales que actuan en caso de especificar que se espera
            una salida con datos estadisticos de los valores en el diccionario '''

        if avarage != None:
            # obtiene el promedio de cada lista asociada a cada clave

            for key in dictio:
                dictio[key] = round(avarage(dictio[key]), 2)

        elif min != None:
            # Obtiene el menor valor de cada lista asociada a cada clave

            for key in dictio:
                dictio[key] = min_value(dictio[key])

        elif counter != None:
            # asocia a cada clave un diccionario con en numero de aparaciones de cada elemento

            for key in dictio.keys():
                dictio[key] = count(dictio[key])


        return dictio

# creacion de los cuatro diccionarios planteados

menor_precio_por_cantidad_habitaciones = create_dict(lower_price_per_bedrooms, bedrooms, price,  string_to_float, min=min_value)
cantidad_reseñas_por_barrio =create_dict(review_per_neigh, neighbourhood, reviews, string_to_float, avarage)
cantidad_tipo_habitacion_por_barrio = create_dict(roomtype_per_neigh, neighbourhood, room, counter=count)
precio_promedio_por_barrio = create_dict(price_per_neigh, neighbourhood, price, string_to_float, avarage)

# Descomentar una linea a vez para visualizar el diccionario en formato mas legible
#   (Se deben correr las celdas anteriores cada vez que quiera imprimir)

# menor_precio_por_cantidad_habitaciones
# cantidad_reseñas_por_barrio
# cantidad_tipo_habitacion_por_barrio
# precio_promedio_por_barrio

# importación de los modulos de gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# aplicamos la utilidad de seaborn
sns.set()

def create_bar(dictio, xlabel=None, ylabel=None, title=None):
    # Crea un grafico de barras

    plt.figure(figsize=(10, 8))

    plt.xticks(rotation=90, fontsize=15)
    plt.yticks(fontsize=15)

    plt.xlabel(xlabel, fontsize=10)
    plt.ylabel(ylabel, fontsize=10)

    plt.title(title, fontsize=20)

    plt.bar(dictio.keys(), dictio.values())

def create_pie(clave, dictio, name=None):
    # crea un grafico de torta

    plt.figure(figsize=(3, 3))
    
    plt.title(name, fontsize=10)

    plt.pie(dictio.values(), labels=dictio.keys())

create_bar(menor_precio_por_cantidad_habitaciones, 'cantidad de habitaciones', 'menor precio', 'menor precio por cantidad de habitaciones')
create_bar(cantidad_reseñas_por_barrio, 'barrios', 'cantidad promedio de reseñas', 'reseñas por barrio')
for barrio, dictio in cantidad_tipo_habitacion_por_barrio.items():
    if barrio in ['Bogenhausen', 'Maxvorstadt', 'Ramersdorf-Perlach', 'Schwanthalerhöhe', 'Ludwigsvorstadt-Isarvorstadt', 'Feldmoching-Hasenbergl']:
      create_pie(barrio, dictio, "tipos de habitaciones en {0}".format(barrio))
create_bar(precio_promedio_por_barrio, 'barrios', 'precio promedio', 'precio promedio por barrio')
